# Исследование всех лингвистических сущностей spaCy

Этот блокнот демонстрирует полные возможности spaCy для извлечения лингвистических сущностей из английского текста:
- Сегментация предложений
- Токенизация (слова, пунктуация)
- Части речи (POS tagging)
- Морфологический анализ
- Лемматизация
- Синтаксический разбор (dependency parsing)
- Распознавание именованных сущностей (NER)
- Извлечение именных групп

**Итоговый результат**: JSON-файл с полной автоматической разметкой текста.

In [1]:
# Импорт необходимых библиотек
import spacy
from spacy.tokens import Doc, Token, Span
import json
from datetime import datetime
from typing import Dict, List, Any
import yaml
import re

print("✓ Библиотеки импортированы")

# Попытка загрузить трансформерную модель (самая точная)
# Если недоступна, используем большую модель
try:
    nlp = spacy.load("en_core_web_trf")
    model_name = "en_core_web_trf (Transformer - наивысшая точность)"
    print(f"✓ Загружена модель: {model_name}")
except OSError:
    try:
        nlp = spacy.load("en_core_web_lg")
        model_name = "en_core_web_lg (Large - высокая точность)"
        print(f"✓ Загружена модель: {model_name}")
    except OSError:
        nlp = spacy.load("en_core_web_sm")
        model_name = "en_core_web_sm (Small - базовая точность)"
        print(f"✓ Загружена модель: {model_name}")

print(f"✓ spaCy версия: {spacy.__version__}")
print(f"✓ Компоненты пайплайна: {nlp.pipe_names}")

✓ Библиотеки импортированы
✓ Загружена модель: en_core_web_trf (Transformer - наивысшая точность)
✓ spaCy версия: 3.8.7
✓ Компоненты пайплайна: ['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## 1. Загрузка и парсинг текста

Загружаем статью из файла и разделяем на:
1. **YAML метаданные** (между `---`)
2. **Основной текст** для обработки spaCy
3. **References** (список литературы в конце)

In [2]:
def parse_document(file_path: str) -> Dict[str, Any]:
    """
    Парсит документ, разделяя на метаданные YAML, основной текст и references.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Извлечение YAML метаданных (между --- в начале)
    yaml_pattern = r'^---\n(.*?)\n---\n'
    yaml_match = re.match(yaml_pattern, content, re.DOTALL)
    
    if yaml_match:
        yaml_content = yaml_match.group(1)
        metadata = yaml.safe_load(yaml_content)
        # Удаляем YAML из основного текста
        remaining_content = content[yaml_match.end():]
    else:
        metadata = {}
        remaining_content = content
    
    # Извлечение References (секция ## References до конца)
    references_pattern = r'\n## References\n+(.*?)$'
    references_match = re.search(references_pattern, remaining_content, re.DOTALL)
    
    if references_match:
        references_text = references_match.group(1)
        main_text = remaining_content[:references_match.start()]
        
        # Парсинг references: [номер]: текст ссылки
        references = {}
        ref_lines = references_text.strip().split('\n')
        for line in ref_lines:
            ref_match = re.match(r'\[(\d+)\]:\s*(.+)$', line)
            if ref_match:
                ref_num = ref_match.group(1)
                ref_text = ref_match.group(2).strip()
                references[ref_num] = ref_text
    else:
        references = {}
        main_text = remaining_content
    
    return {
        'metadata': metadata,
        'main_text': main_text.strip(),
        'references': references
    }

# Загрузка и парсинг документа
file_path = r'd:\Knowledge_Map\prompts\Новый формат статьи. Первый этап.eng.md'
parsed_doc = parse_document(file_path)

print(f"✓ Документ загружен: {file_path}")
print(f"✓ Метаданных: {len(parsed_doc['metadata'])} полей")
print(f"✓ Длина основного текста: {len(parsed_doc['main_text'])} символов")
print(f"✓ Количество references: {len(parsed_doc['references'])}")
print(f"\n--- Превью метаданных ---")
print(f"DOI: {parsed_doc['metadata'].get('doi', 'N/A')}")
print(f"Journal: {parsed_doc['metadata'].get('journal', 'N/A')}")
print(f"Authors: {list(parsed_doc['metadata'].get('authors', {}).keys())[:3]}...")
print(f"\n--- Превью текста (первые 200 символов) ---")
print(parsed_doc['main_text'][:200] + "...")

✓ Документ загружен: d:\Knowledge_Map\prompts\Новый формат статьи. Первый этап.eng.md
✓ Метаданных: 11 полей
✓ Длина основного текста: 41177 символов
✓ Количество references: 120

--- Превью метаданных ---
DOI: 10.1111/febs.12335
Journal: The FEBS
Authors: ['Paul M. A. Antony', 'Nico J. Diederich', 'Rejko Krüger']...

--- Превью текста (первые 200 символов) ---
# The hallmarks of Parkinson's disease

## Abstract

Since the discovery of dopamine as a neurotransmitter in the 1950s, Parkinson's disease (PD) research has generated a rich and complex body of know...


## 2. Функции извлечения лингвистических признаков

Создаём функции для извлечения всех доступных признаков из spaCy:
- **Токены**: 20+ атрибутов (текст, лемма, POS, морфология, синтаксис, семантика)
- **Предложения**: токены, зависимости, именные группы
- **Документ**: полная иерархическая структура

In [3]:
def extract_token_features(token: Token) -> Dict[str, Any]:
    """
    Извлекает ВСЕ доступные лингвистические признаки из токена spaCy.
    
    Включает:
    - Базовые атрибуты (текст, лемма, форма)
    - Части речи (универсальные и детальные)
    - Морфологические признаки (время, вид, залог, число, род, падеж и др.)
    - Синтаксические связи (главное слово, зависимые)
    - Именованные сущности
    - Булевы флаги (is_alpha, is_punct, is_stop и др.)
    - Позиционная информация
    """
    
    # Извлечение морфологических признаков
    morphology = {}
    if token.morph:
        for key in ["PronType", "Number", "Case", "Gender", "Tense", 
                    "Aspect", "Mood", "Voice", "Person", "Degree", 
                    "VerbForm", "Poss", "Reflex", "NumType", "Foreign"]:
            values = token.morph.get(key)
            if values:
                morphology[key] = values[0] if len(values) == 1 else list(values)
    
    # Извлечение информации о сущностях
    entity_info = None
    if token.ent_type_:
        entity_info = {
            "type": token.ent_type_,
            "iob": token.ent_iob_
        }
    
    return {
        # Базовые текстовые атрибуты
        "id": token.i,
        "text": token.text,
        "text_with_ws": token.text_with_ws,
        "lemma": token.lemma_,
        "lower": token.lower_,
        "shape": token.shape_,
        
        # Части речи
        "pos": token.pos_,           # Универсальный POS тег
        "tag": token.tag_,           # Детальный POS тег
        
        # Синтаксис
        "dep": token.dep_,           # Синтаксическая роль (зависимость)
        "head_id": token.head.i,     # ID главного слова
        "head_text": token.head.text,
        "children_ids": [child.i for child in token.children],
        "children_texts": [child.text for child in token.children],
        "n_lefts": token.n_lefts,    # Количество левых зависимых
        "n_rights": token.n_rights,  # Количество правых зависимых
        
        # Морфология
        "morphology": morphology if morphology else None,
        
        # Именованные сущности
        "entity": entity_info,
        
        # Булевы флаги
        "flags": {
            "is_alpha": token.is_alpha,
            "is_ascii": token.is_ascii,
            "is_digit": token.is_digit,
            "is_lower": token.is_lower,
            "is_upper": token.is_upper,
            "is_title": token.is_title,
            "is_punct": token.is_punct,
            "is_space": token.is_space,
            "is_stop": token.is_stop,
            "like_num": token.like_num,
            "like_url": token.like_url,
            "like_email": token.like_email,
        },
        
        # Позиционная информация
        "position": {
            "token_index": token.i,
            "char_start": token.idx,
            "char_end": token.idx + len(token.text)
        }
    }

print("✓ Функция extract_token_features() создана")
print("  Извлекает: текст, лемма, POS, морфология, синтаксис, сущности, флаги, позиция")

✓ Функция extract_token_features() создана
  Извлекает: текст, лемма, POS, морфология, синтаксис, сущности, флаги, позиция


In [4]:
def extract_sentence_features(sent: Span, sent_id: int) -> Dict[str, Any]:
    """
    Извлекает все признаки уровня предложения.
    
    Включает:
    - Текст предложения и границы
    - Все токены с полными признаками
    - Синтаксические зависимости (граф head-dependent)
    - Именные группы (noun chunks)
    """
    
    # Извлечение всех токенов
    tokens = [extract_token_features(token) for token in sent]
    
    # Извлечение синтаксических зависимостей
    dependencies = []
    for token in sent:
        if token.dep_ != "ROOT":  # ROOT не имеет главного слова
            dependencies.append({
                "dependent_id": token.i,
                "dependent_text": token.text,
                "head_id": token.head.i,
                "head_text": token.head.text,
                "relation": token.dep_
            })
    
    # Извлечение именных групп
    noun_chunks = []
    for chunk in sent.noun_chunks:
        noun_chunks.append({
            "text": chunk.text,
            "root_text": chunk.root.text,
            "root_id": chunk.root.i,
            "start_token": chunk.start,
            "end_token": chunk.end,
            "start_char": chunk.start_char,
            "end_char": chunk.end_char
        })
    
    return {
        "id": sent_id,
        "text": sent.text,
        "start_char": sent.start_char,
        "end_char": sent.end_char,
        "start_token": sent.start,
        "end_token": sent.end,
        "tokens": tokens,
        "dependencies": dependencies,
        "noun_chunks": noun_chunks,
        "num_tokens": len(tokens),
        "num_dependencies": len(dependencies),
        "num_noun_chunks": len(noun_chunks)
    }

print("✓ Функция extract_sentence_features() создана")
print("  Извлекает: токены, зависимости, именные группы")

✓ Функция extract_sentence_features() создана
  Извлекает: токены, зависимости, именные группы


In [5]:
def process_document_with_spacy(text: str, metadata: Dict, references: Dict) -> Dict[str, Any]:
    """
    Обрабатывает документ с помощью spaCy и создаёт полную JSON-структуру.
    
    Структура выхода:
    - metadata: метаданные из YAML
    - references: список литературы
    - processing_info: информация об обработке
    - document: лингвистическая разметка текста
    """
    
    print("Обработка текста с помощью spaCy...")
    doc = nlp(text)
    
    # Извлечение предложений
    print("Извлечение предложений и токенов...")
    sentences = [extract_sentence_features(sent, i) for i, sent in enumerate(doc.sents)]
    
    # Извлечение всех именованных сущностей
    print("Извлечение именованных сущностей...")
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "type": ent.label_,
            "start_char": ent.start_char,
            "end_char": ent.end_char,
            "start_token": ent.start,
            "end_token": ent.end
        })
    
    # Извлечение всех именных групп на уровне документа
    print("Извлечение именных групп...")
    all_noun_chunks = []
    for chunk in doc.noun_chunks:
        # Найти предложение, которому принадлежит chunk
        sent_id = None
        for i, sent in enumerate(doc.sents):
            if chunk.start >= sent.start and chunk.end <= sent.end:
                sent_id = i
                break
        
        all_noun_chunks.append({
            "text": chunk.text,
            "root_text": chunk.root.text,
            "root_id": chunk.root.i,
            "sentence_id": sent_id,
            "start_token": chunk.start,
            "end_token": chunk.end,
            "start_char": chunk.start_char,
            "end_char": chunk.end_char
        })
    
    # Статистика частей речи
    print("Подсчёт статистики...")
    pos_counts = {}
    for token in doc:
        pos = token.pos_
        pos_counts[pos] = pos_counts.get(pos, 0) + 1
    
    # Статистика типов сущностей
    entity_type_counts = {}
    for ent in entities:
        ent_type = ent["type"]
        entity_type_counts[ent_type] = entity_type_counts.get(ent_type, 0) + 1
    
    # Формирование итоговой структуры
    result = {
        "metadata": metadata,
        "references": references,
        "processing_info": {
            "model": nlp.meta["name"],
            "model_version": nlp.meta["version"],
            "spacy_version": spacy.__version__,
            "processed_at": datetime.now().isoformat(),
            "statistics": {
                "text_length": len(text),
                "num_sentences": len(sentences),
                "num_tokens": len(doc),
                "num_entities": len(entities),
                "num_noun_chunks": len(all_noun_chunks),
                "pos_distribution": pos_counts,
                "entity_type_distribution": entity_type_counts
            }
        },
        "document": {
            "text": text,
            "sentences": sentences,
            "entities": entities,
            "noun_chunks": all_noun_chunks
        }
    }
    
    print(f"✓ Обработка завершена!")
    print(f"  - Предложений: {len(sentences)}")
    print(f"  - Токенов: {len(doc)}")
    print(f"  - Сущностей: {len(entities)}")
    print(f"  - Именных групп: {len(all_noun_chunks)}")
    
    return result

print("✓ Функция process_document_with_spacy() создана")
print("  Создаёт полную JSON-структуру с метаданными, references и лингвистической разметкой")

✓ Функция process_document_with_spacy() создана
  Создаёт полную JSON-структуру с метаданными, references и лингвистической разметкой


## 3. Обработка документа

Запускаем полную обработку текста статьи о болезни Паркинсона.

In [6]:
# Обработка документа
annotated_document = process_document_with_spacy(
    text=parsed_doc['main_text'],
    metadata=parsed_doc['metadata'],
    references=parsed_doc['references']
)

print("\n" + "="*60)
print("СТАТИСТИКА ОБРАБОТКИ")
print("="*60)
stats = annotated_document['processing_info']['statistics']
print(f"Длина текста: {stats['text_length']:,} символов")
print(f"Предложений: {stats['num_sentences']}")
print(f"Токенов: {stats['num_tokens']:,}")
print(f"Именованных сущностей: {stats['num_entities']}")
print(f"Именных групп: {stats['num_noun_chunks']}")

print(f"\nРаспределение частей речи (топ-10):")
pos_sorted = sorted(stats['pos_distribution'].items(), key=lambda x: x[1], reverse=True)
for pos, count in pos_sorted[:10]:
    print(f"  {pos:10s}: {count:4d}")

print(f"\nТипы именованных сущностей:")
for ent_type, count in sorted(stats['entity_type_distribution'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {ent_type:15s}: {count:3d}")

Обработка текста с помощью spaCy...
Извлечение предложений и токенов...
Извлечение именованных сущностей...
Извлечение именных групп...
Подсчёт статистики...
✓ Обработка завершена!
  - Предложений: 290
  - Токенов: 7711
  - Сущностей: 92
  - Именных групп: 1550

СТАТИСТИКА ОБРАБОТКИ
Длина текста: 41,177 символов
Предложений: 290
Токенов: 7,711
Именованных сущностей: 92
Именных групп: 1550

Распределение частей речи (топ-10):
  NOUN      : 1704
  PUNCT     : 1039
  ADP       :  794
  ADJ       :  753
  X         :  749
  VERB      :  531
  DET       :  495
  AUX       :  264
  PROPN     :  251
  ADV       :  224

Типы именованных сущностей:
  CARDINAL       :  47
  DATE           :  12
  PERSON         :  10
  ORDINAL        :   9
  PERCENT        :   7
  MONEY          :   6
  PRODUCT        :   1


## 4. Экспорт в JSON

Сохраняем полную лингвистическую разметку в JSON-файл.

In [7]:
# Сохранение в JSON
output_path = r'd:\Knowledge_Map\notebooks\spacy_annotations_output.json'

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(annotated_document, f, ensure_ascii=False, indent=2)

print(f"✓ JSON-файл сохранён: {output_path}")
print(f"  Размер файла: {len(json.dumps(annotated_document, ensure_ascii=False)):,} байт")

# Проверка структуры JSON
print("\n" + "="*60)
print("СТРУКТУРА JSON")
print("="*60)
print(f"Верхний уровень ключей: {list(annotated_document.keys())}")
print(f"\nmetadata содержит: {list(annotated_document['metadata'].keys())}")
print(f"references содержит: {len(annotated_document['references'])} ссылок")
print(f"processing_info содержит: {list(annotated_document['processing_info'].keys())}")
print(f"document содержит: {list(annotated_document['document'].keys())}")

✓ JSON-файл сохранён: d:\Knowledge_Map\notebooks\spacy_annotations_output.json
  Размер файла: 6,380,996 байт

СТРУКТУРА JSON
Верхний уровень ключей: ['metadata', 'references', 'processing_info', 'document']

metadata содержит: ['doi', 'journal', 'article_type', 'authors', 'affiliations', 'keywords', 'correspondence', 'article_stages', 'abbreviations', 'lince', 'page_sub_info']
references содержит: 120 ссылок
processing_info содержит: ['model', 'model_version', 'spacy_version', 'processed_at', 'statistics']
document содержит: ['text', 'sentences', 'entities', 'noun_chunks']


## 5. Исследование результатов

Посмотрим на примеры извлечённой лингвистической информации.

In [8]:
# Пример 1: Первое предложение с полной разметкой
print("="*80)
print("ПРИМЕР 1: ПЕРВОЕ ПРЕДЛОЖЕНИЕ С ПОЛНОЙ РАЗМЕТКОЙ")
print("="*80)

first_sent = annotated_document['document']['sentences'][0]
print(f"\nПредложение #{first_sent['id']}:")
print(f"Текст: {first_sent['text']}")
print(f"Границы символов: {first_sent['start_char']} - {first_sent['end_char']}")
print(f"Количество токенов: {first_sent['num_tokens']}")
print(f"Количество зависимостей: {first_sent['num_dependencies']}")
print(f"Количество именных групп: {first_sent['num_noun_chunks']}")

print(f"\n--- Токены с признаками ---")
for token in first_sent['tokens'][:5]:  # Первые 5 токенов
    print(f"\nТокен #{token['id']}: '{token['text']}'")
    print(f"  Лемма: {token['lemma']}")
    print(f"  POS: {token['pos']} (детальный: {token['tag']})")
    print(f"  Синтаксическая роль: {token['dep']}")
    print(f"  Главное слово: '{token['head_text']}' (id={token['head_id']})")
    if token['morphology']:
        print(f"  Морфология: {token['morphology']}")
    if token['entity']:
        print(f"  Сущность: {token['entity']['type']} ({token['entity']['iob']})")
    print(f"  Флаги: alpha={token['flags']['is_alpha']}, punct={token['flags']['is_punct']}, stop={token['flags']['is_stop']}")

print(f"\n--- Синтаксические зависимости (первые 5) ---")
for dep in first_sent['dependencies'][:5]:
    print(f"'{dep['dependent_text']}' --[{dep['relation']}]--> '{dep['head_text']}'")

if first_sent['noun_chunks']:
    print(f"\n--- Именные группы ---")
    for chunk in first_sent['noun_chunks']:
        print(f"  '{chunk['text']}' (корень: '{chunk['root_text']}')")

ПРИМЕР 1: ПЕРВОЕ ПРЕДЛОЖЕНИЕ С ПОЛНОЙ РАЗМЕТКОЙ

Предложение #0:
Текст: # The hallmarks of Parkinson's disease

## Abstract
Границы символов: 0 - 51
Количество токенов: 11
Количество зависимостей: 10
Количество именных групп: 3

--- Токены с признаками ---

Токен #0: '#'
  Лемма: #
  POS: PUNCT (детальный: NFP)
  Синтаксическая роль: punct
  Главное слово: 'hallmarks' (id=2)
  Флаги: alpha=False, punct=True, stop=False

Токен #1: 'The'
  Лемма: the
  POS: DET (детальный: DT)
  Синтаксическая роль: det
  Главное слово: 'hallmarks' (id=2)
  Морфология: {'PronType': 'Art'}
  Флаги: alpha=True, punct=False, stop=True

Токен #2: 'hallmarks'
  Лемма: hallmark
  POS: NOUN (детальный: NNS)
  Синтаксическая роль: ROOT
  Главное слово: 'hallmarks' (id=2)
  Морфология: {'Number': 'Plur'}
  Флаги: alpha=True, punct=False, stop=False

Токен #3: 'of'
  Лемма: of
  POS: ADP (детальный: IN)
  Синтаксическая роль: prep
  Главное слово: 'hallmarks' (id=2)
  Флаги: alpha=True, punct=False, stop=True

Ток

In [9]:
# Пример 2: Именованные сущности
print("\n" + "="*80)
print("ПРИМЕР 2: ИМЕНОВАННЫЕ СУЩНОСТИ (первые 20)")
print("="*80)

entities = annotated_document['document']['entities']
for ent in entities[:20]:
    print(f"{ent['type']:15s} | {ent['text']}")


ПРИМЕР 2: ИМЕНОВАННЫЕ СУЩНОСТИ (первые 20)
DATE            | the 1950s
ORDINAL         | second
CARDINAL        | 1
PERCENT         | ~ 1%
DATE            | older than 60 years
DATE            | the age of 80 years
PERCENT         | 3%
PERCENT         | 5-10%
DATE            | the early 1990s
DATE            | 15 years
ORDINAL         | first
CARDINAL        | ~ 28
CARDINAL        | only six
PERCENT         | approximately 30-70%
DATE            | age 80 years
CARDINAL        | 18
CARDINAL        | 1
CARDINAL        | 1
CARDINAL        | Four
CARDINAL        | 150


In [10]:
# Пример 3: Примеры синтаксических отношений
print("\n" + "="*80)
print("ПРИМЕР 3: ПРИМЕРЫ СИНТАКСИЧЕСКИХ ОТНОШЕНИЙ")
print("="*80)

# Собираем примеры разных типов зависимостей
dep_examples = {}
for sent in annotated_document['document']['sentences']:
    for dep in sent['dependencies']:
        rel = dep['relation']
        if rel not in dep_examples:
            dep_examples[rel] = []
        if len(dep_examples[rel]) < 2:  # По 2 примера каждого типа
            dep_examples[rel].append(f"'{dep['dependent_text']}' --> '{dep['head_text']}'")

# Показываем примеры
for rel in sorted(dep_examples.keys())[:15]:  # Первые 15 типов
    examples = ", ".join(dep_examples[rel])
    print(f"{rel:15s} : {examples}")


ПРИМЕР 3: ПРИМЕРЫ СИНТАКСИЧЕСКИХ ОТНОШЕНИЙ
acl             : 'influenced' --> 'disease', 'targeting' --> 'endeavors'
acomp           : 'symptomatic' --> 'is', 'tempting' --> 'is'
advcl           : 'revealing' --> 'generated', 'organizing' --> 'propose'
advmod          : 'second' --> 'common', 'most' --> 'common'
agent           : 'by' --> 'influenced', 'by' --> 'increased'
amod            : 'rich' --> 'body', 'related' --> 'disease'
appos           : 'Abstract' --> 'hallmarks', 'IPD' --> 'disease'
attr            : 'disease' --> 'be', 'disease' --> 'is'
aux             : 'has' --> 'generated', 'to' --> 'be'
auxpass         : 'is' --> 'increased', 'is' --> 'characterized'
case            : ''s' --> 'Parkinson', ''s' --> 'Parkinson'
cc              : 'and' --> 'rich', 'and' --> 'genetic'
ccomp           : 'be' --> 'revealing', 'experience' --> 'start'
compound        : 'disease' --> 'hallmarks', 'research' --> 'endeavors'
conj            : 'complex' --> 'rich', 'environmental' --> 'gene

In [11]:
# Пример 4: Морфологический анализ
print("\n" + "="*80)
print("ПРИМЕР 4: МОРФОЛОГИЧЕСКИЙ АНАЛИЗ")
print("="*80)

# Ищем токены с интересной морфологией
print("Глаголы с временем и видом:")
verb_count = 0
for sent in annotated_document['document']['sentences']:
    for token in sent['tokens']:
        if token['pos'] == 'VERB' and token['morphology']:
            morph = token['morphology']
            if 'Tense' in morph or 'Aspect' in morph:
                tense = morph.get('Tense', '-')
                aspect = morph.get('Aspect', '-')
                voice = morph.get('Voice', '-')
                print(f"  '{token['text']}' (лемма: {token['lemma']}) - Tense:{tense}, Aspect:{aspect}, Voice:{voice}")
                verb_count += 1
                if verb_count >= 10:
                    break
    if verb_count >= 10:
        break

print("\nСуществительные с числом:")
noun_count = 0
for sent in annotated_document['document']['sentences']:
    for token in sent['tokens']:
        if token['pos'] == 'NOUN' and token['morphology']:
            morph = token['morphology']
            if 'Number' in morph:
                number = morph.get('Number', '-')
                print(f"  '{token['text']}' - Number:{number}")
                noun_count += 1
                if noun_count >= 10:
                    break
    if noun_count >= 10:
        break


ПРИМЕР 4: МОРФОЛОГИЧЕСКИЙ АНАЛИЗ
Глаголы с временем и видом:
  'generated' (лемма: generate) - Tense:Past, Aspect:Perf, Voice:-
  'revealing' (лемма: reveal) - Tense:Pres, Aspect:Prog, Voice:-
  'related' (лемма: relate) - Tense:Past, Aspect:Perf, Voice:-
  'influenced' (лемма: influence) - Tense:Past, Aspect:Perf, Voice:-
  'increased' (лемма: increase) - Tense:Past, Aspect:Perf, Voice:-
  'explore' (лемма: explore) - Tense:Pres, Aspect:-, Voice:-
  'propose' (лемма: propose) - Tense:Pres, Aspect:-, Voice:-
  'based' (лемма: base) - Tense:Past, Aspect:Perf, Voice:-
  'organizing' (лемма: organize) - Tense:Pres, Aspect:Prog, Voice:-
  'encourage' (лемма: encourage) - Tense:Pres, Aspect:-, Voice:-

Существительные с числом:
  'hallmarks' - Number:Plur
  'disease' - Number:Sing
  'Abstract' - Number:Sing
  'discovery' - Number:Sing
  'dopamine' - Number:Sing
  'neurotransmitter' - Number:Sing
  '1950s' - Number:Plur
  'disease' - Number:Sing
  'research' - Number:Sing
  'body' - Number:

In [13]:
# Пример 5: Визуализация синтаксического дерева (опционально)
print("\n" + "="*80)
print("ПРИМЕР 5: ВИЗУАЛИЗАЦИЯ С DISPLACY")
print("="*80)

from spacy import displacy

# Берём короткое предложение для визуализации
sample_text = "Parkinson's disease is the second most common neurodegenerative disease."
doc = nlp(sample_text)

print(f"Предложение для визуализации: {sample_text}\n")

# Показываем dependency parse в текстовом виде
for token in doc:
    print(f"{token.text:20s} | {token.pos_:8s} | {token.dep_:12s} | --> {token.head.text}")

# Сохранение визуализации в HTML (без использования jupyter)
# Используем jupyter=False чтобы избежать ошибки импорта IPython.display
try:
    html = displacy.render(doc, style="dep", page=True, jupyter=False)
    html_path = r'd:\Knowledge_Map\notebooks\spacy_dependency_visualization.html'
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html)
    print(f"\n✓ Визуализация зависимостей сохранена в: {html_path}")
except Exception as e:
    print(f"\n⚠ Не удалось создать HTML-визуализацию: {e}")

# Визуализация именованных сущностей
try:
    html_ner = displacy.render(doc, style="ent", page=True, jupyter=False)
    html_ner_path = r'd:\Knowledge_Map\notebooks\spacy_entities_visualization.html'
    with open(html_ner_path, 'w', encoding='utf-8') as f:
        f.write(html_ner)
    print(f"✓ Визуализация сущностей сохранена в: {html_ner_path}")
except Exception as e:
    print(f"⚠ Не удалось создать HTML-визуализацию сущностей: {e}")


ПРИМЕР 5: ВИЗУАЛИЗАЦИЯ С DISPLACY
Предложение для визуализации: Parkinson's disease is the second most common neurodegenerative disease.

Parkinson            | PROPN    | poss         | --> disease
's                   | PART     | case         | --> Parkinson
disease              | NOUN     | nsubj        | --> is
is                   | AUX      | ROOT         | --> is
the                  | DET      | det          | --> disease
second               | ADJ      | amod         | --> common
most                 | ADV      | advmod       | --> common
common               | ADJ      | amod         | --> disease
neurodegenerative    | ADJ      | amod         | --> disease
disease              | NOUN     | attr         | --> is
.                    | PUNCT    | punct        | --> is

✓ Визуализация зависимостей сохранена в: d:\Knowledge_Map\notebooks\spacy_dependency_visualization.html
✓ Визуализация сущностей сохранена в: d:\Knowledge_Map\notebooks\spacy_entities_visualization.html


## Заключение

В этом блокноте мы исследовали **все возможности spaCy** для извлечения лингвистических сущностей из английского текста:

### Извлечённые лингвистические признаки:

**На уровне токена:**
- Базовые: текст, лемма, нижний регистр, форма (shape)
- Части речи: универсальный POS (17 категорий), детальный тег
- Морфология: время, вид, залог, наклонение, число, род, падеж, лицо, степень и др.
- Синтаксис: синтаксическая роль (50+ типов зависимостей), главное слово, зависимые слова
- Семантика: тип именованной сущности, IOB-теги
- Булевы флаги: 12 флагов (is_alpha, is_punct, is_stop, like_num и др.)
- Позиция: индекс токена, смещение символов

**На уровне предложения:**
- Границы предложения (символы, токены)
- Все токены с полными признаками
- Граф синтаксических зависимостей
- Именные группы с корнями

**На уровне документа:**
- Метаданные из YAML
- Список литературы (references)
- Все предложения
- Глобальный список именованных сущностей
- Глобальный список именных групп
- Статистика: распределение POS, типы сущностей

### Результаты:
- ✅ JSON-файл с полной автоматической разметкой: `spacy_annotations_output.json`
- ✅ HTML-визуализация синтаксического дерева: `spacy_dependency_visualization.html`

### Применение:
Этот подход можно использовать для:
- Лингвистического анализа научных текстов
- Извлечения структурированной информации
- Построения графов знаний
- Обучения моделей машинного обучения
- Анализа синтаксических паттернов

## Справочник лингвистических сущностей spaCy


### Универсальные части речи (Universal POS Tags)

<table>
<thead>
<tr>
<th>Код</th>
<th>English</th>
<th>Русский</th>
</tr>
</thead>
<tbody>
<tr><td>ADJ</td><td>Adjective</td><td>Прилагательное</td></tr>
<tr><td>ADP</td><td>Adposition</td><td>Предлог</td></tr>
<tr><td>ADV</td><td>Adverb</td><td>Наречие</td></tr>
<tr><td>AUX</td><td>Auxiliary</td><td>Вспомогательный глагол</td></tr>
<tr><td>CCONJ</td><td>Coordinating Conjunction</td><td>Сочинительный союз</td></tr>
<tr><td>DET</td><td>Determiner</td><td>Определитель</td></tr>
<tr><td>INTJ</td><td>Interjection</td><td>Междометие</td></tr>
<tr><td>NOUN</td><td>Noun</td><td>Существительное</td></tr>
<tr><td>NUM</td><td>Numeral</td><td>Числительное</td></tr>
<tr><td>PART</td><td>Particle</td><td>Частица</td></tr>
<tr><td>PRON</td><td>Pronoun</td><td>Местоимение</td></tr>
<tr><td>PROPN</td><td>Proper Noun</td><td>Имя собственное</td></tr>
<tr><td>PUNCT</td><td>Punctuation</td><td>Пунктуация</td></tr>
<tr><td>SCONJ</td><td>Subordinating Conjunction</td><td>Подчинительный союз</td></tr>
<tr><td>SYM</td><td>Symbol</td><td>Символ</td></tr>
<tr><td>VERB</td><td>Verb</td><td>Глагол</td></tr>
<tr><td>X</td><td>Other</td><td>Прочее</td></tr>
<tr><td>SPACE</td><td>Space</td><td>Пробел</td></tr>
</tbody>
</table>

### Синтаксические зависимости (Dependency Relations)

<table>
<thead>
<tr>
<th>Код</th>
<th>English</th>
<th>Русский</th>
</tr>
</thead>
<tbody>
<tr><td>acl</td><td>Clausal Modifier of Noun</td><td>Придаточное определение</td></tr>
<tr><td>acomp</td><td>Adjectival Complement</td><td>Адъективное дополнение</td></tr>
<tr><td>advcl</td><td>Adverbial Clause Modifier</td><td>Обстоятельственное придаточное</td></tr>
<tr><td>advmod</td><td>Adverbial Modifier</td><td>Обстоятельство</td></tr>
<tr><td>agent</td><td>Agent</td><td>Агенс (действующее лицо)</td></tr>
<tr><td>amod</td><td>Adjectival Modifier</td><td>Определение (прилагательное)</td></tr>
<tr><td>appos</td><td>Appositional Modifier</td><td>Приложение</td></tr>
<tr><td>attr</td><td>Attribute</td><td>Именная часть сказуемого</td></tr>
<tr><td>aux</td><td>Auxiliary</td><td>Вспомогательный глагол</td></tr>
<tr><td>auxpass</td><td>Passive Auxiliary</td><td>Вспомогательный глагол страдательного залога</td></tr>
<tr><td>case</td><td>Case Marking</td><td>Падежный показатель</td></tr>
<tr><td>cc</td><td>Coordinating Conjunction</td><td>Сочинительный союз</td></tr>
<tr><td>ccomp</td><td>Clausal Complement</td><td>Придаточное дополнение</td></tr>
<tr><td>compound</td><td>Compound</td><td>Сложное слово</td></tr>
<tr><td>conj</td><td>Conjunct</td><td>Однородный член</td></tr>
<tr><td>csubj</td><td>Clausal Subject</td><td>Придаточное подлежащее</td></tr>
<tr><td>csubjpass</td><td>Clausal Passive Subject</td><td>Придаточное подлежащее (страдательный залог)</td></tr>
<tr><td>dative</td><td>Dative</td><td>Дательный падеж</td></tr>
<tr><td>dep</td><td>Unspecified Dependency</td><td>Неопределённая зависимость</td></tr>
<tr><td>det</td><td>Determiner</td><td>Определитель</td></tr>
<tr><td>dobj</td><td>Direct Object</td><td>Прямое дополнение</td></tr>
<tr><td>expl</td><td>Expletive</td><td>Формальное подлежащее</td></tr>
<tr><td>intj</td><td>Interjection</td><td>Междометие</td></tr>
<tr><td>mark</td><td>Marker</td><td>Маркер (союз)</td></tr>
<tr><td>meta</td><td>Meta Modifier</td><td>Мета-модификатор</td></tr>
<tr><td>neg</td><td>Negation Modifier</td><td>Отрицание</td></tr>
<tr><td>nmod</td><td>Nominal Modifier</td><td>Именной модификатор</td></tr>
<tr><td>npadvmod</td><td>Noun Phrase as Adverbial Modifier</td><td>Именная группа в роли обстоятельства</td></tr>
<tr><td>nsubj</td><td>Nominal Subject</td><td>Подлежащее</td></tr>
<tr><td>nsubjpass</td><td>Passive Nominal Subject</td><td>Подлежащее (страдательный залог)</td></tr>
<tr><td>nummod</td><td>Numeric Modifier</td><td>Числовой модификатор</td></tr>
<tr><td>oprd</td><td>Object Predicate</td><td>Объектный предикат</td></tr>
<tr><td>parataxis</td><td>Parataxis</td><td>Паратаксис (сочинительная связь)</td></tr>
<tr><td>pcomp</td><td>Complement of Preposition</td><td>Дополнение предлога</td></tr>
<tr><td>pobj</td><td>Object of Preposition</td><td>Объект предлога</td></tr>
<tr><td>poss</td><td>Possession Modifier</td><td>Притяжательный модификатор</td></tr>
<tr><td>preconj</td><td>Preconjunct</td><td>Предсоюз</td></tr>
<tr><td>predet</td><td>Predeterminer</td><td>Предопределитель</td></tr>
<tr><td>prep</td><td>Prepositional Modifier</td><td>Предложное дополнение</td></tr>
<tr><td>prt</td><td>Particle</td><td>Частица</td></tr>
<tr><td>punct</td><td>Punctuation</td><td>Пунктуация</td></tr>
<tr><td>quantmod</td><td>Modifier of Quantifier</td><td>Модификатор квантора</td></tr>
<tr><td>relcl</td><td>Relative Clause Modifier</td><td>Относительное придаточное</td></tr>
<tr><td>ROOT</td><td>Root</td><td>Корень (главное слово предложения)</td></tr>
<tr><td>xcomp</td><td>Open Clausal Complement</td><td>Инфинитивное дополнение</td></tr>
</tbody>
</table>

### Типы именованных сущностей (Named Entity Types)

<table>
<thead>
<tr>
<th>Код</th>
<th>English</th>
<th>Русский</th>
</tr>
</thead>
<tbody>
<tr><td>PERSON</td><td>People, including fictional</td><td>Люди, включая вымышленных персонажей</td></tr>
<tr><td>NORP</td><td>Nationalities, Religious, Political groups</td><td>Национальности, религиозные и политические группы</td></tr>
<tr><td>FAC</td><td>Buildings, Airports, Highways, Bridges</td><td>Здания, аэропорты, шоссе, мосты</td></tr>
<tr><td>ORG</td><td>Companies, Agencies, Institutions</td><td>Компании, агентства, учреждения</td></tr>
<tr><td>GPE</td><td>Countries, Cities, States</td><td>Страны, города, штаты</td></tr>
<tr><td>LOC</td><td>Non-GPE Locations, Mountain ranges, Bodies of water</td><td>Горные хребты, водоёмы, не-GPE локации</td></tr>
<tr><td>PRODUCT</td><td>Objects, Vehicles, Foods (not services)</td><td>Объекты, транспорт, продукты (не услуги)</td></tr>
<tr><td>EVENT</td><td>Named Hurricanes, Battles, Wars, Sports events</td><td>Ураганы, битвы, войны, спортивные события</td></tr>
<tr><td>WORK_OF_ART</td><td>Titles of Books, Songs, etc.</td><td>Названия книг, песен и т.д.</td></tr>
<tr><td>LAW</td><td>Named Documents made into Laws</td><td>Законы и правовые документы</td></tr>
<tr><td>LANGUAGE</td><td>Any Named Language</td><td>Любой именованный язык</td></tr>
<tr><td>DATE</td><td>Absolute or Relative Dates or Periods</td><td>Абсолютные или относительные даты/периоды</td></tr>
<tr><td>TIME</td><td>Times smaller than a Day</td><td>Время (меньше дня)</td></tr>
<tr><td>PERCENT</td><td>Percentage (including "%")</td><td>Проценты (включая "%")</td></tr>
<tr><td>MONEY</td><td>Monetary Values (including unit)</td><td>Денежные суммы (с единицами)</td></tr>
<tr><td>QUANTITY</td><td>Measurements (weight, distance, etc.)</td><td>Измерения (вес, расстояние и т.д.)</td></tr>
<tr><td>ORDINAL</td><td>First, Second, etc.</td><td>Порядковые числительные (первый, второй и т.д.)</td></tr>
<tr><td>CARDINAL</td><td>Numerals that do not fall under another type</td><td>Количественные числительные</td></tr>
</tbody>
</table>

### Морфологические признаки (Morphological Features)

<table>
<thead>
<tr>
<th>Признак</th>
<th>Значения</th>
<th>Русский</th>
</tr>
</thead>
<tbody>
<tr><td>Tense</td><td>Past, Pres, Fut</td><td>Время: прошедшее, настоящее, будущее</td></tr>
<tr><td>Aspect</td><td>Perf, Prog, Imp</td><td>Вид: совершенный, продолженный, несовершенный</td></tr>
<tr><td>Mood</td><td>Ind, Imp, Sub</td><td>Наклонение: изъявительное, повелительное, сослагательное</td></tr>
<tr><td>Voice</td><td>Act, Pass</td><td>Залог: действительный, страдательный</td></tr>
<tr><td>Number</td><td>Sing, Plur</td><td>Число: единственное, множественное</td></tr>
<tr><td>Person</td><td>1, 2, 3</td><td>Лицо: первое, второе, третье</td></tr>
<tr><td>Gender</td><td>Masc, Fem, Neut</td><td>Род: мужской, женский, средний</td></tr>
<tr><td>Case</td><td>Nom, Acc, Gen, Dat</td><td>Падеж: именительный, винительный, родительный, дательный</td></tr>
<tr><td>Degree</td><td>Pos, Cmp, Sup</td><td>Степень: положительная, сравнительная, превосходная</td></tr>
<tr><td>VerbForm</td><td>Fin, Inf, Part, Ger</td><td>Форма глагола: финитная, инфинитив, причастие, герундий</td></tr>
<tr><td>PronType</td><td>Prs, Art, Dem, Rel</td><td>Тип местоимения: личное, артикль, указательное, относительное</td></tr>
<tr><td>Poss</td><td>Yes</td><td>Притяжательное</td></tr>
<tr><td>Reflex</td><td>Yes</td><td>Возвратное</td></tr>
<tr><td>NumType</td><td>Card, Ord</td><td>Тип числительного: количественное, порядковое</td></tr>
</tbody>
</table>